In [ ]:
#default_exp clean

# Data Cleaning

<br>

### Imports

In [ ]:
#exports
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

import os
import glob
from ipypb import track

from batopt import utils, retrieval

In [ ]:
from IPython.display import JSON

<br>

### User Inputs

In [ ]:
raw_data_dir = '../data/raw'
cache_data_dir = '../data/nb-cache'

<br>

### Loading the Raw Data

We'll start by loading in the demand data, first we have to determine the latest training set that is available for us to work with

In [ ]:
#exports
def identify_latest_set_num(data_dir):
    set_num = max([
        int(f.split('_set')[1].replace('.csv', '')) 
        for f in os.listdir(data_dir) 
        if 'set' in f
    ])
    
    return set_num

In [ ]:
set_num = identify_latest_set_num(raw_data_dir)

set_num

<br>

We'll then load in and clean the datetime index of the dataset

In [ ]:
#exports
def reindex_df_dt_idx(df, freq='30T'):
    full_dt_idx = pd.date_range(df.index.min(), df.index.max(), freq=freq)
    df = df.reindex(full_dt_idx)
    
    return df

def load_training_dataset(raw_data_dir: str, dataset_name: str='demand', set_num=None, parse_dt_idx: bool=True, dt_idx_freq: str='30T') -> pd.DataFrame:
    if set_num is None:
        set_num = identify_latest_set_num(raw_data_dir)
        
    allowed_datasets = ['demand', 'pv', 'weather']
    assert dataset_name in allowed_datasets, f"`dataset_name` must be one of: {', '.join(allowed_datasets)} - not {dataset_name}"
    
    df = pd.read_csv(glob.glob(f'{raw_data_dir}/{dataset_name}*set{set_num}.csv')[0].replace('\\', '/'))
    
    if parse_dt_idx == True:
        assert 'datetime' in df.columns, 'if `parse_dt_idx` is True then `datetime` must be a column in the dataset'
        
        df['datetime'] = pd.to_datetime(df['datetime'], utc=True)
        df = df.set_index('datetime').pipe(reindex_df_dt_idx, freq=dt_idx_freq).sort_index(axis=1)
        df.index.name = 'datetime'

    return df

In [ ]:
df_demand = load_training_dataset(raw_data_dir, 'demand')

df_demand.head()

<br>

Then the pv

In [ ]:
df_pv = load_training_dataset(raw_data_dir, 'pv')

df_pv.head()

<br>

And finally the weather

In [ ]:
df_weather = load_training_dataset(raw_data_dir, 'weather', dt_idx_freq='H')

df_weather.head(3)

<br>

We'll also create a function that reads all of the datasets in at once and then combines them

In [ ]:
#exports
def combine_training_datasets(raw_data_dir, set_num=None):
    # Loading provided training datasets
    single_datasets = dict()
    dataset_names = ['demand', 'pv', 'weather']

    for dataset_name in dataset_names:
        single_datasets[dataset_name] = load_training_dataset(raw_data_dir, dataset_name, set_num=set_num)
    
    # Constructing date range
    min_dt = min([df.index.min() for df in single_datasets.values()])
    max_dt = max([df.index.max() for df in single_datasets.values()]) + pd.Timedelta(minutes=30)

    dt_rng = pd.date_range(min_dt, max_dt, freq='30T')
    
    # Constructing combined dataframe
    df_combined = pd.DataFrame(index=dt_rng, columns=dataset_names)
    
    for dataset_name in dataset_names:
        df_single_dataset = single_datasets[dataset_name]
        cols_to_be_overwritten = set(df_combined.columns) - (set(df_combined.columns) - set(df_single_dataset.columns))
        assert len(cols_to_be_overwritten) == 0, f"The following columns exist in multiple datasets meaning data would be overwritten: {', '.join(cols_to_be_overwritten)}"

        df_combined[df_single_dataset.columns] = df_single_dataset
    
    df_combined = df_combined.sort_index()
    
    # Adding holiday dates
    s_holidays = retrieval.load_holidays_s(raw_data_dir)
    
    s_cropped_holidays = s_holidays[max(df_combined.index.min(), s_holidays.index.min()):
                                    min(df_combined.index.max(), s_holidays.index.max())]
    
    df_combined.loc[s_cropped_holidays.index, 'holidays'] = s_cropped_holidays
    
    return df_combined

In [ ]:
df_combined = combine_training_datasets(raw_data_dir)

df_combined.head(3)


<br>

### Identifying Missing Values

We'll quickly inspect the datasets and check their coverage over the full date range when aggregated by dataset

In [ ]:
#exports
def identify_df_dt_entries(df_demand, df_pv, df_weather):
    min_dt = min(df_demand.index.min(), df_pv.index.min(), df_weather.index.min())
    max_dt = max(df_demand.index.max(), df_pv.index.max(), df_weather.index.max())
    
    dt_rng = pd.date_range(min_dt, max_dt, freq='30T')
    df_nulls = pd.DataFrame(index=dt_rng)
    
    df_nulls['demand'] = df_demand.reindex(dt_rng).isnull().mean(axis=1).astype(int)
    df_nulls['pv'] = df_pv.reindex(dt_rng).isnull().mean(axis=1).astype(int)
    df_nulls['weather'] = df_weather.reindex(dt_rng).ffill(limit=1).isnull().mean(axis=1).astype(int)
    
    df_entries = 1 - df_nulls
    
    return df_entries

In [ ]:
df_entries = identify_df_dt_entries(df_demand, df_pv, df_weather)

# Plotting
fig, ax = plt.subplots(dpi=150)

sns.heatmap(df_entries.T, ax=ax, cmap=plt.cm.binary)

utils.set_date_ticks(ax, '2015-01-01', '2018-07-31', axis='x', freq='Qs', date_format='%b %y')

<br>

We'll also determine the null percentage in each individual column

In [ ]:
df_demand.isnull().mean()

<br>

We can see that all of the PV data columns are missing some data

In [ ]:
df_pv.isnull().mean()

<br>

Location 2 is also missing some solar data

In [ ]:
df_weather.isnull().mean()

In [ ]:
# !!! We also need to handle the two days where demand is constantly 0 !!!
# Separately it looks like there's also some anomalous 0 readings for the PV output

In [ ]:
(df_demand==0).sum()

In [ ]:
df_demand.query('demand_MW==0').head()

<br>

### Handling Missing Values

We'll start by interpolating the missing PV data, first checking the number of variables that have null values for each time period

In [ ]:
s_pv_num_null_vals = df_pv.isnull().sum(axis=1).replace(0, np.nan).dropna().astype(int)

s_pv_num_null_vals.value_counts()

<br>

`pv_power_mw` and `irradiance_Wm-2` have the same average number of null values, there are also no time-periods where only 2 variables have null values - it's therefore likely that power and irradiance always have null periods at the same time which makes it harder to interpolate their values. We'll quickly check this hypothesis:

In [ ]:
(df_pv['pv_power_mw'].isnull() == df_pv['irradiance_Wm-2'].isnull()).mean() == 1

<br>

It appears as though the `pv_power_mw` and `irradiance_Wm-2` missing values are a single time-block that coincides with a larger set of missing values within `panel_temp_C`.

In [ ]:
df_pv[df_pv['pv_power_mw'].isnull()]

<br>

Looking at the `panel_temp_C` data we can see there are 3 time-blocks where obervations are missing 

In [ ]:
df_pv['panel_temp_C'].isnull().astype(int).plot()

<br>

One option might to be replace the missing temperature values with the temperatures observed at the surrounding weather grid locations, we'll start by constructing a dataframe that includes all of the temperature data as well as the average rolling temperature for each weather data location.

In [ ]:
#exports
def construct_df_temp_features(df_weather, df_pv):
    df_weather = df_weather.reindex(pd.date_range(df_weather.index.min(), df_weather.index.max(), freq='30T')).ffill(limit=1)
    temp_loc_cols = df_weather.columns[df_weather.columns.str.contains('temp')]
    
    df_temp_features = (df_weather
                        .copy()
                        [temp_loc_cols]
                        .assign(site_temp=df_pv['panel_temp_C'])
                       )

    df_temp_features[[col+'_rolling' for col in temp_loc_cols]] = df_temp_features.rolling(3).mean()[temp_loc_cols]

    df_temp_features = df_temp_features.sort_index(axis=1)

    return df_temp_features

In [ ]:
df_temp_features = construct_df_temp_features(df_weather, df_pv).dropna()

df_temp_features.head()

<br>

We'll now check the correlation

In [ ]:
sns.heatmap(df_temp_features.corr())

<br>

The correlation drops off quickly when it gets to the site temperature, looking at the full distributions we can see that the site measurements get far higher. This is because the panel is absorbing heat that raises its temperature above that of the surrounding area, again making it more difficult to simply fill in with the nearby temperature measurements.

In [ ]:
sns.distplot(df_temp_features['site_temp'], color='C0', label='Panel')
sns.distplot(df_temp_features.drop('site_temp', axis=1).min(axis=1), color='C1', label='MERRA Min')
sns.distplot(df_temp_features.drop('site_temp', axis=1).max(axis=1), color='C2', label='MERRA Max')

plt.legend(frameon=False)

In [ ]:
# Could use an RF to estimate the panel temp based on the weather grid temps?
# Potential features: current average surrounding temp, average surrounding temp over the last 3 hours

In [ ]:
#exports
def split_X_y_data(df, target_col='site_temp'):
    df = df.dropna()
    X_cols = df.drop(target_col, axis=1).columns

    X = df[X_cols].values
    y = df[target_col].values
    
    return X, y

def split_X_y_data_with_index(df, target_col='site_temp'):
    df = df.dropna()
    X_cols = df.drop(target_col, axis=1).columns

    X = df[X_cols].values
    y = df[target_col].values
    index = df.index
    
    return X, y, index

In [ ]:
X, y = split_X_y_data(df_temp_features)

X.shape, y.shape

In [ ]:
#exports
def generate_kfold_preds(
    X, 
    y, 
    model=LinearRegression(), 
    kfold_kwargs={'n_splits': 5, 'shuffle': True},
    index=None
):

    kfold = KFold(**kfold_kwargs)
    df_pred = pd.DataFrame(columns=['pred', 'true'], index=np.arange(X.shape[0]))

    for train_idxs, test_idxs in kfold.split(X):
        X_train, y_train = X[train_idxs], y[train_idxs]
        X_test, y_test = X[test_idxs], y[test_idxs]
    
        model.fit(X_train, y_train)
        
        df_pred.loc[test_idxs, 'true'] = y_test
        df_pred.loc[test_idxs, 'pred'] = model.predict(X_test)
        
    df_pred = df_pred.sort_index()
    
    if index is not None:
        assert len(index) == df_pred.shape[0], 'The passed index must be the same length as X and y'
        df_pred.index = index
        
    return df_pred

In [ ]:
df_pred = generate_kfold_preds(X, y)

df_pred.head()

In [ ]:
#exports
def evaluate_models(X, y, models, post_pred_proc_func=None, index=None):
    model_scores = dict()

    for model_name, model in track(models.items()):
        df_pred = generate_kfold_preds(X, y, model, index=index)
        
        if post_pred_proc_func is not None:
            df_pred['pred'] = post_pred_proc_func(df_pred['pred'])

        model_scores[model_name] = {
            'mae': mean_absolute_error(df_pred['true'], df_pred['pred']),
            'rmse': np.sqrt(mean_squared_error(df_pred['true'], df_pred['pred']))
        }

    df_model_scores = pd.DataFrame(model_scores)
    
    df_model_scores.index.name = 'metric'
    df_model_scores.columns.name = 'model'

    return df_model_scores

In [ ]:
models = {
    'std_linear': LinearRegression(),
    'random_forest': RandomForestRegressor(),
    'boosted': GradientBoostingRegressor()
}

rerun_panel_temp_model = True
model_scores_filename = 'panel_temp_interp_model_results.csv'

if (rerun_panel_temp_model == True) or (model_scores_filename not in os.listdir(cache_data_dir)):
    df_model_scores = evaluate_models(X, y, models)
    df_model_scores.to_csv(f'{cache_data_dir}/{model_scores_filename}')
else:
    df_model_scores = pd.read_csv(f'{cache_data_dir}/{model_scores_filename}', index_col='metric')

df_model_scores

In [ ]:
top_model = df_model_scores.T['rmse'].idxmin()
df_pred = generate_kfold_preds(X, y, models[top_model])

df_pred.head()

In [ ]:
s_residuals = df_pred.diff(1, axis=1).dropna(axis=1).iloc[:, 0]

s_residuals.plot(linewidth=0.3)

In [ ]:
plt.scatter(df_pred['true'], df_pred['pred'], s=1)

plt.xlabel('Obervation')
plt.ylabel('Prediction')

In [ ]:
#exports
def interpolate_missing_panel_temps(df_pv, df_weather, model=RandomForestRegressor()):
    missing_panel_temp_dts = df_pv.index[df_pv['panel_temp_C'].isnull()]
    
    if len(missing_panel_temp_dts) == 0: # i.e. no missing values
        return df_pv

    df_temp_features = construct_df_temp_features(df_weather, df_pv)
    missing_dt_X = df_temp_features.loc[missing_panel_temp_dts].drop('site_temp', axis=1).values
    X, y = split_X_y_data(df_temp_features, 'site_temp')
    
    model.fit(X, y)
    df_pv.loc[missing_panel_temp_dts, 'panel_temp_C'] = model.predict(missing_dt_X)
    
    assert df_pv['panel_temp_C'].isnull().sum() == 0, 'There are still null values for the PV panel temperature'
    
    return df_pv

In [ ]:
df_pv = interpolate_missing_panel_temps(df_pv, df_weather)

df_pv.isnull().mean()

In [ ]:
#exports
def construct_df_irradiance_features(df_weather, df_pv):
    df_weather = df_weather.reindex(pd.date_range(df_weather.index.min(), df_weather.index.max(), freq='30T')).ffill(limit=1)
    temp_loc_cols = df_weather.columns[df_weather.columns.str.contains('solar')]
    
    df_irradiance_features = (df_weather
                              .copy()
                              [temp_loc_cols]
                              .assign(site_solar=df_pv['irradiance_Wm-2'])
                              .pipe(lambda df: df.assign(hour=df.index.hour + df.index.minute/60))
                             )

    df_irradiance_features = df_irradiance_features.sort_index(axis=1)

    return df_irradiance_features

In [ ]:
df_irradiance_features = construct_df_irradiance_features(df_weather, df_pv)

df_irradiance_features.head()

In [ ]:
models = {
    'std_linear': LinearRegression(),
    'random_forest': RandomForestRegressor(),
    'boosted': GradientBoostingRegressor()
}

rerun_site_irradiance_model = True
model_scores_filename = 'site_irradiance_interp_model_results.csv'

X, y = split_X_y_data(df_irradiance_features, 'site_solar')

if (rerun_site_irradiance_model == True) or (model_scores_filename not in os.listdir(cache_data_dir)):
    df_model_scores = evaluate_models(X, y, models)
    df_model_scores.to_csv(f'{cache_data_dir}/{model_scores_filename}')
else:
    df_model_scores = pd.read_csv(f'{cache_data_dir}/{model_scores_filename}', index_col='metric')

df_model_scores

In [ ]:
top_model = df_model_scores.T['rmse'].idxmin()
df_pred = generate_kfold_preds(X, y, models[top_model])

df_pred.head()

In [ ]:
plt.scatter(df_pred['true'], df_pred['pred'], s=1)

plt.xlabel('Obervation')
plt.ylabel('Prediction')

In [ ]:
#exports
def interpolate_missing_site_irradiance(df_pv, df_weather, model=RandomForestRegressor()):
    missing_site_irradiance_dts = df_pv.index[df_pv['irradiance_Wm-2'].isnull()]
    
    if len(missing_site_irradiance_dts) == 0: # i.e. no missing values
        return df_pv

    df_irradiance_features = construct_df_irradiance_features(df_weather, df_pv)
    missing_dt_X = df_irradiance_features.loc[missing_site_irradiance_dts].drop('site_solar', axis=1).values
    X, y = split_X_y_data(df_irradiance_features, 'site_solar')
 
    model.fit(X, y)
    df_pv.loc[missing_site_irradiance_dts, 'irradiance_Wm-2'] = model.predict(missing_dt_X)
    
    assert df_pv['irradiance_Wm-2'].isnull().sum() == 0, 'There are still null values for the solar site irradiance'
    
    return df_pv

In [ ]:
df_pv = interpolate_missing_site_irradiance(df_pv, df_weather)

df_pv.isnull().mean()

<br>

Now that we have the irradiance and temperature we're ready to start filling in the missing values for power output, again using the same regression interpolation method

In [ ]:
#exports
def construct_df_power_features(df_pv):
    df_power_features = (df_pv
                         .pipe(lambda df: df.assign(hour=df.index.hour + df.index.minute/60))
                         .sort_index(axis=1)
                        )

    return df_power_features

In [ ]:
df_power_features = construct_df_power_features(df_pv)

df_power_features.head()

In [ ]:
models = {
    'std_linear': LinearRegression(),
    'random_forest': RandomForestRegressor(),
    'boosted': GradientBoostingRegressor()
}

rerun_site_power_model = True
model_scores_filename = 'site_power_interp_model_results.csv'

X, y, dates = split_X_y_data_with_index(df_power_features, 'pv_power_mw')

if (rerun_site_power_model == True) or (model_scores_filename not in os.listdir(cache_data_dir)):
    df_model_scores = evaluate_models(X, y, models)
    df_model_scores.to_csv(f'{cache_data_dir}/{model_scores_filename}')
else:
    df_model_scores = pd.read_csv(f'{cache_data_dir}/{model_scores_filename}', index_col='metric')

df_model_scores

In [ ]:
top_model = df_model_scores.T['rmse'].idxmin()
df_pred = generate_kfold_preds(X, y, models[top_model])

df_pred.head()

In [ ]:
plt.scatter(df_pred['true'], df_pred['pred'], s=1)

plt.xlabel('Obervation')
plt.ylabel('Prediction')

##### Anomalous data points in PV data

The PV data shows a number of points where the observed value is 0 but the prediction is much higher. 

First let's try and identify them (setting the tolerance to be lower will capture more values as anomalous). 

In [ ]:
def identify_anomalies_pv(df_pred, tolerance=0.1):
    foo = df_pred.copy()
    foo['difference'] = foo.pred - foo.true
    foo = foo[(foo.difference > tolerance) & (foo.true == 0)]
    return foo.index

anomalous_dates = dates[identify_anomalies_pv(df_pred)]
anomalous_df = df_power_features[df_power_features.index.isin(anomalous_dates)]
plt.hist(anomalous_df.hour) # Check this histogram to eyeball if any unreasonable anomalous values are caught by the tolerance (e.g. late at night)

Replace these values in `df_power_features`. 

In [ ]:
df_power_features_clean = df_power_features.copy()
df_power_features_clean.loc[df_power_features_clean.index.isin(anomalous_dates), 'pv_power_mw'] = np.nan

Rerun the previous model fitting and check the pred vs. actual graph. 

In [ ]:
models = {
    'std_linear': LinearRegression(),
    'random_forest': RandomForestRegressor(),
    'boosted': GradientBoostingRegressor()
}

rerun_site_power_model = True
model_scores_filename = 'site_power_interp_clean_model_results.csv'

X, y, dates = split_X_y_data_with_index(df_power_features_clean, 'pv_power_mw')

if (rerun_site_power_model == True) or (model_scores_filename not in os.listdir(cache_data_dir)):
    df_model_scores = evaluate_models(X, y, models)
    df_model_scores.to_csv(f'{cache_data_dir}/{model_scores_filename}')
else:
    df_model_scores = pd.read_csv(f'{cache_data_dir}/{model_scores_filename}', index_col='metric')

top_model = df_model_scores.T['rmse'].idxmin()
df_pred = generate_kfold_preds(X, y, models[top_model])

plt.scatter(df_pred['true'], df_pred['pred'], s=1)

plt.xlabel('Obervation')
plt.ylabel('Prediction')

The above graph looks to be a cleaner with tolerance at 0.1. It looks like there might still be some which aren't though. Consider lowering the tolerance.

In [ ]:
#exports
def pv_anomalies_to_nan(df_pv, model=GradientBoostingRegressor(), tolerance=0.1):
    """
    Run this function to identify places where predicted values for pv_power_mw are much larger
    than true values and where the true value is 0 (we expect these are anomalies) and make these values nan.

    """
    df_power_features = construct_df_power_features(df_pv)
    
    X, y, dates = split_X_y_data_with_index(df_power_features, 'pv_power_mw')
    df_pred = generate_kfold_preds(X, y, model)
    df_pred['difference'] = df_pred.pred - df_pred.true
    df_pred['datetime'] = dates
    df_pred = df_pred.set_index('datetime')
    
    anomalous_idx = df_pred[(df_pred.difference > tolerance) & (df_pred.true == 0)].index    
    
    df_pv.loc[df_pv.index.isin(anomalous_idx), 'pv_power_mw'] = np.nan
        
    return df_pv

In [ ]:
df_pv = pv_anomalies_to_nan(df_pv)

In [ ]:
#exports
def interpolate_missing_site_power(df_pv, model=RandomForestRegressor()):
    missing_site_power_dts = df_pv.index[df_pv['pv_power_mw'].isnull()]
    
    if len(missing_site_power_dts) == 0: # i.e. no missing values
        return df_pv

    df_power_features = construct_df_power_features(df_pv)
    missing_dt_X = df_power_features.loc[missing_site_power_dts].drop('pv_power_mw', axis=1).values
    X, y = split_X_y_data(df_power_features, 'pv_power_mw')
 
    model.fit(X, y)
    df_pv.loc[missing_site_power_dts, 'pv_power_mw'] = model.predict(missing_dt_X)
    
    assert df_pv['pv_power_mw'].isnull().sum() == 0, 'There are still null values for the solar site power'
    
    return df_pv

In [ ]:
df_pv = interpolate_missing_site_power(df_pv)

df_pv.isnull().mean()

In [ ]:
#exports
def interpolate_missing_weather_solar(df_pv, df_weather, weather_col='solar_location2', model=RandomForestRegressor()):
    missing_weather_solar_dts = df_weather.index[df_weather[weather_col].isnull()]
    
    if len(missing_weather_solar_dts) == 0: # i.e. no missing values
        return df_pv

    df_irradiance_features = construct_df_irradiance_features(df_weather, df_pv).drop('site_solar', axis=1)
    missing_dt_X = df_irradiance_features.loc[missing_weather_solar_dts].drop(weather_col, axis=1).values
    X, y = split_X_y_data(df_irradiance_features, weather_col)
 
    model.fit(X, y)
    df_weather.loc[missing_weather_solar_dts, weather_col] = model.predict(missing_dt_X)
    
    assert df_weather[weather_col].isnull().sum() == 0, 'There are still null values for the weather dataset solar observations'
    
    return df_weather

In [ ]:
df_weather = interpolate_missing_weather_solar(df_pv, df_weather, model=LinearRegression())

df_weather.isnull().mean()

<br>

### Interpolate Missing Temp Data



In [ ]:
#export
def interpolate_missing_temps(df_weather, temp_variable, model=RandomForestRegressor()):
    """
    Use the other temperature locations to predict missing values of `temp_variable`. 
    
    For test_2: a full day is missing for temp_location4 on 2018-11-18
    """    
    missing_temp_dts = df_weather.index[df_weather[temp_variable].isnull()]
        
    if len(missing_temp_dts) == 0: # i.e. no missing values
        return df_weather
    
    temp_loc_cols = df_weather.columns[df_weather.columns.str.contains('temp')]

    df_temp_features = df_weather.filter(temp_loc_cols)
    missing_dt_X = df_temp_features.loc[missing_temp_dts].drop(temp_variable, axis=1).values
        
    X, y = split_X_y_data(df_temp_features, temp_variable)
        
    model.fit(X, y)
    
    df_weather.loc[missing_temp_dts, temp_variable] = model.predict(missing_dt_X)
    df_weather = df_weather.ffill(limit=1)
            
    assert df_weather[temp_variable].isnull().sum() == 0, 'There are still null values for the PV panel temperature'
    
    return df_weather

In [ ]:
foo = interpolate_missing_temps(df_weather, 'temp_location4', model=LinearRegression())
foo.isnull().mean()

<br>

Finally we'll export the relevant code to our `batopt` module

In [ ]:
#hide
from nbdev.export import notebook2script
    
notebook2script()